In [3]:
using Pkg
using CSV 
using DataFrames
using Dates

In [4]:
using Statistics
using Random

In [5]:
filename = "/u/sauves/leucegene-shared/Data/LEUCEGENE/lgn_pronostic_GE_CDS_TPM.csv"
#GE_TRSC_TPM = DataFrame(CSV.File(filename))
@time GE_CDS_TPM = CSV.read(filename, DataFrame)
print()

 10.134341 seconds (24.32 M allocations: 1.168 GiB, 4.40% gc time)


In [1]:
mutable struct Data
    name::String
    data::Array
    factor_1::Array
    factor_2::Array
end 

In [2]:
mutable struct FoldData 
    name::String 
    train::Data
    test::Data
end 

In [33]:
index = GE_CDS_TPM[:,1]
data = GE_CDS_TPM[:,2:end] 
cols = names(data)
# log transforming
data = log10.(Array(data) .+ 1)
# remove least varying genes
ge_var = var(data,dims = 1) 
ge_var_med = median(ge_var)
# high variance only 
hvg = getindex.(findall(ge_var .> ge_var_med),2)
data = data[:,hvg]
cols = cols[hvg]
# verify that the operation worked
sum(var(data, dims = 1) .< ge_var_med)
# split into train test
nsamples = length(index)
indices = shuffle(Array{Int}(1:nsamples))
nfolds = 5
foldsize = Int(nsamples / 5)

folds = Array{FoldData}(undef,5)
for i in 1:nfolds
    tst_idx = indices[(i - 1) * foldsize + 1: i * foldsize]
    tr_idx = setdiff(indices, tst_idx)
    test = Data("test", data[tst_idx,:], index[tst_idx], cols)
    train = Data("train", data[tr_idx,:], index[tr_idx], cols)
    fold_data = FoldData("fold_$i", test, train)
    folds[i] = fold_data
end
folds[1].train.factor_1
folds[1].train.data




60×9798 Matrix{Float64}:
 0.251476    0.816532  0.0792985  0.0959853  …  1.67931     3.86236  3.5624
 0.203714    0.812801  0.10696    0.441416      0.00935652  4.0532   3.86567
 0.362471    0.741427  0.0983849  0.048334      1.75403     3.74081  3.49489
 0.194721    1.119     0.0764072  0.417699      0.0117825   3.72051  3.75596
 0.509079    0.675201  0.114725   0.32963       0.01916     3.79279  3.46841
 0.646923    0.599898  0.0170589  0.137031   …  0.0         3.77769  3.80402
 0.481917    0.827749  0.207171   0.146581      1.93889     3.94721  3.76362
 0.250592    0.575862  0.296395   0.230795      1.4306      3.78853  3.59275
 0.0203769   1.0523    0.371108   0.492245      0.0         3.32057  3.51836
 0.623365    1.06335   0.102529   0.413146      1.53726     3.99022  3.5371
 ⋮                                           ⋱  ⋮                    
 0.00700229  0.866796  0.0248754  0.221988      0.0         3.72776  3.33528
 0.736662    0.594659  0.0591334  0.12302       1.66703     